In [1]:
import json

with open('dataset/LPI12 dataset.json', 'r') as f:
        TrainData = json.load(f)

In [3]:
import models._config as c

from dataset.RadarDataset import RadarSignalDataset

train_dataset = RadarSignalDataset(TrainData, c.signalTypes[0:c.typeSize], snr_max=17)


Data loading for 'Barker'.....Done!
Data loading for 'Costas'.....Done!
Data loading for 'Frank'.....Done!
Data loading for 'LFM'.....Done!
Data loading for 'P1'.....Done!
Data loading for 'P2'.....Done!
Data loading for 'P3'.....Done!
Data loading for 'P4'.....Done!
Data loading for 'T1'.....Done!
Data loading for 'T2'.....Done!
Data loading for 'T3'.....Done!
Data loading for 'T4'.....Done!


: 

In [2]:

import matplotlib.pyplot as plt

# 각 클래스별로 몇 개의 신호를 시각화할 것인지 설정
samples_per_class = 1

# 클래스별로 데이터 인덱스를 저장할 딕셔너리 초기화
class_samples = {signal_type: [] for signal_type in set(train_dataset.labels)}

# 각 클래스별로 필요한 수만큼의 샘플 수집
for idx in range(len(train_dataset)):
    _, label, _, _ = train_dataset[idx]
    if len(class_samples[label]) < samples_per_class:
        class_samples[label].append(idx)
    # 모든 클래스의 샘플 수집이 완료되면 종료
    if all(len(samples) == samples_per_class for samples in class_samples.values()):
        break

# 각 클래스별로 수집한 샘플을 시각화
for label, indices in class_samples.items():
    fig, axes = plt.subplots(len(indices), 2, figsize=(12, 3 * len(indices)))
    fig.suptitle(f'Signal Visualization for Class: {label}', fontsize=16)

    for i, idx in enumerate(indices):
        data_sample, _, snr = train_dataset[idx]
        
        # 실수부와 허수부 추출
        real_part = [point[0] for point in data_sample]
        imag_part = [point[1] for point in data_sample]

        # 실수부 플롯
        axes[i, 0].plot(real_part)
        axes[i, 0].set_title(f'Real Part (SNR: {snr} dB)')
        axes[i, 0].set_xlabel('Time Step')
        axes[i, 0].set_ylabel('Amplitude')

        # 허수부 플롯
        axes[i, 1].plot(imag_part)
        axes[i, 1].set_title(f'Imaginary Part (SNR: {snr} dB)')
        axes[i, 1].set_xlabel('Time Step')
        axes[i, 1].set_ylabel('Amplitude')

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()
    




KeyboardInterrupt: 